In [10]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import requests
from bs4 import BeautifulSoup

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False  # 음수 부호 깨짐 방지

# 'app_manager/data' 폴더가 없으면 생성
data_folder = os.path.join('app_manager', 'data')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# headers 설정
headers = {"User-Agent": "Mozilla/5.0"}

# 종목 코드와 기업명 매핑
stock_info = {
    '현대차': '005380',
    '기아': '000270',
    '에코프로비엠': '247540',
    'LG화학': '051910',
    '금양': '001570',
    'SK이노베이션': '096770',
    'LG 에너지솔루션': '373220',
    '삼성SDI': '006400',
    '엘앤에프': '066970',
    '포스코퓨처엠': '003670',
    '포스코DX': '022100',
    'SK 네트웍스': '001740',
    '솔루엠': '248070',
    '롯데이노베이트': '286940',
}

def get_data(stock_code, end_date=None, pages=1):
    data = []  # 데이터를 저장할 리스트
    
    for page in range(1, pages + 1):
        base_url = f'https://finance.naver.com/item/sise_day.nhn?code={stock_code}'  # 종목별 URL 설정
        params = {
            "code": stock_code,
            "page": page
        }
        try:
            res = requests.get(base_url, headers=headers, params=params)
            res.encoding = 'euc-kr'  # 네이버 금융은 EUC-KR 인코딩 사용
            soup = BeautifulSoup(res.text, "html.parser")
            
            table = soup.find("table", {"class": "type2"})
            if not table:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                continue
            
            # 테이블에서 행(row) 추출
            rows = table.find_all("tr")
            
            for row in rows:
                cols = row.find_all("td")
                if len(cols) > 1:  # 공백 row 제외
                    try:
                        date = cols[0].get_text(strip=True)
                        closing_price = cols[1].get_text(strip=True).replace(',', '')
                        volume = cols[6].get_text(strip=True).replace(',', '')
                        
                        # 종료일이 설정되어 있다면 그 날짜 이전의 데이터만 추가
                        parsed_date = pd.to_datetime(date, format='%Y.%m.%d')
                        if end_date and parsed_date <= pd.to_datetime(end_date):
                            print(f"종목 {stock_code}: 종료일({end_date}) 이전의 데이터는 제외됩니다.")
                            return pd.DataFrame(data)

                        data.append({
                            '날짜': date,
                            '종가': int(closing_price) if closing_price else None,
                            '거래량': int(volume) if volume else None
                        })
                    except ValueError as ve:
                        print(f"데이터 변환 오류: {ve}")
        except requests.RequestException as e:
            print(f"HTTP 요청 중 오류 발생: {e}")
    
    # 리스트를 DataFrame으로 변환
    df = pd.DataFrame(data)

    # 날짜 형식 변환
    try:
        df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d')
    except Exception as e:
        print(f"날짜 변환 중 오류 발생: {e}")
        return df  # 오류 발생 시에도 데이터프레임 반환
    
    # NaT 값 제거
    df.dropna(subset=['날짜'], inplace=True)
    
    # 정렬
    df.sort_values('날짜', inplace=True, ascending=False)
    
    # 인덱스 재설정
    df.reset_index(drop=True, inplace=True)
    
    return df

# 각 종목의 데이터를 가져와 기업별로 CSV 파일로 저장
end_date = "2022-01-01"  # 종료일 설정
for company_name, stock_code in stock_info.items():

    # 종목의 데이터를 가져옴
    stock_data = get_data(stock_code=stock_code, end_date=end_date, pages=68)
    
    # 기업명을 파일 이름에 포함하여 CSV 저장
    csv_filename = os.path.join(data_folder, f'{company_name}_daily_prices.csv')
    stock_data.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"{csv_filename} 파일이 저장되었습니다.")

    # 날짜를 인덱스로 설정
    stock_data.set_index('날짜', inplace=True)

    # NaN 또는 NaT가 있는 데이터를 제외
    stock_data = stock_data.dropna(subset=['종가'])
    
    # 그래프 생성
    plt.figure(figsize=(12, 8))
    plt.plot(stock_data.index, stock_data['종가'], linestyle='-', color='green', label='종가')
    
    # x축을 분기 단위로 설정
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # 3개월 간격으로 라벨 표시
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 연도-월 형식으로 표시
    
    # 그래프 제목 및 레이블 설정
    plt.title(f'{company_name} 일별 종가 그래프', fontsize=15)
    plt.xlabel('날짜 (분기별)', fontsize=12)
    plt.ylabel('종가 가격 (KRW)', fontsize=12)
    
    # x축의 날짜 라벨이 겹치지 않도록 회전
    plt.xticks(rotation=45)
    
    # 범례 표시
    plt.legend()

    # 그래프 저장
    graph_filename = os.path.join(data_folder, f'{company_name}_price_trend.png')
    plt.savefig(graph_filename)
    print(f"{graph_filename} 그래프 파일이 저장되었습니다.")

    # 그래프 초기화 (다음 종목을 위해)
    plt.clf()

print("모든 CSV 파일과 그래프 파일 저장 완료.")



app_manager\data\현대차_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\현대차_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\기아_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\기아_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\에코프로비엠_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\에코프로비엠_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\LG화학_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\LG화학_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\금양_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\금양_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\SK이노베이션_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\SK이노베이션_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\LG 에너지솔루션_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\LG 에너지솔루션_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\삼성SDI_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\삼성SDI_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\엘앤에프_daily_prices.csv 파일이 저장되었습니다.
app_manager\data\엘앤에프_price_trend.png 그래프 파일이 저장되었습니다.
app_manager\data\포스코퓨처엠_d

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

...

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>